# Build Classification Model

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve
from sklearn.svm import SVC
import pandas as pd
import numpy as np

In [48]:
transformed_feature_df = pd.read_csv(".data/features_dataset.csv")
transformed_feature_df= transformed_feature_df.drop(['Unnamed: 0'], axis=1)
transformed_feature_df.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [49]:
transformed_label_df = pd.read_csv(".data/labels_dataset.csv")
transformed_label_df= transformed_label_df.drop(['Unnamed: 0'], axis=1)
transformed_label_df.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


In [50]:
X_train, X_test, y_train, y_test = train_test_split(transformed_feature_df, transformed_label_df, test_size=0.3)

In [59]:
lr = LogisticRegression(multi_class='ovr',solver='lbfgs')
model = lr.fit(X_train, np.ravel(y_train))

accuracy = model.score(X_test, y_test)
print ("Accuracy is {}".format(accuracy))

Accuracy is 0.8031693077564637


In [52]:
# test an item
print(f'ingredients: {X_test.iloc[20][X_test.iloc[20]!=0].keys()}')
print(f'cusine: {y_test.iloc[20]}')

ingredients: Index(['corn'], dtype='object')
cusine: cuisine    thai
Name: 3816, dtype: object


In [53]:
#rehsape to 2d array and transpose
test= X_test.iloc[20].values.reshape(-1, 1).T
# predict with score
proba = model.predict_proba(test)
classes = model.classes_
# create df with classes and scores
resultdf = pd.DataFrame(data=proba, columns=classes)

# create df to show results
topPrediction = resultdf.T.sort_values(by=[0], ascending = [False])
topPrediction.head()

,0
thai,0.475724
chinese,0.201912
japanese,0.152046
korean,0.110980
indian,0.059338


In [54]:
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.77      0.70      0.74       239
      indian       0.88      0.88      0.88       240
    japanese       0.76      0.79      0.77       227
      korean       0.86      0.78      0.82       240
        thai       0.75      0.86      0.80       253

    accuracy                           0.80      1199
   macro avg       0.81      0.80      0.80      1199
weighted avg       0.81      0.80      0.80      1199



# Try different classifiers

In [60]:

C = 10
# Create different classifiers.
classifiers = {
    'L1 logistic': LogisticRegression(C=C, penalty='l1',
                                      solver='saga',
                                      multi_class='multinomial',
                                      max_iter=10000),
    'L2 logistic (Multinomial)': LogisticRegression(C=C, penalty='l2',
                                                    solver='saga',
                                                    multi_class='multinomial',
                                                    max_iter=10000),
    'L2 logistic (OvR)': LogisticRegression(C=C, penalty='l2',
                                            solver='saga',
                                            multi_class='ovr',
                                            max_iter=10000),
    'Linear SVC': SVC(kernel='linear', C=C, probability=True,
                      random_state=0)
}


In [61]:
n_classifiers = len(classifiers)

for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X_train, np.ravel(y_train))

    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))


Accuracy (train) for L1 logistic: 79.9% 
Accuracy (train) for L2 logistic (Multinomial): 79.7% 
Accuracy (train) for L2 logistic (OvR): 79.8% 
Accuracy (train) for Linear SVC: 77.9% 
